In [ ]:
import pandas as pd
import os
from geopy.distance import geodesic

# 서울의 위도/경도 범위 (대략적인 서울 경계)
SEOUl_MIN_LAT, SEOUl_MAX_LAT = 37.4138, 37.7151
SEOUl_MIN_LON, SEOUl_MAX_LON = 126.7341, 127.2686

# 파일 경로와 범위
base_input_path = '/content/drive/MyDrive/여의도_불꽃축제_데이터/원본 데이터/'
base_output_path = '/content/drive/MyDrive/여의도_불꽃축제_데이터/결측값 제거 데이터/'

# 폴더가 없으면 생성
os.makedirs(base_output_path, exist_ok=True)

# 파일들을 처리
for i in range(13):
    input_file_path = f"{base_input_path}KSR_SJUNIV_TRIP_YYD_{i:012d}.csv"
    output_file_path = f"{base_output_path}KSR_SJUNIV_TRIP_YYD_{i:012d}_cleaned.csv"


    # 파일 읽기
    df = pd.read_csv(input_file_path)

    # 결측값 및 잘못된 좌표 제거
    def valid_row(row):
        # 출발좌표나 도착좌표가 (0, 0)인 경우는 넘어감
        if row['DPR_X_AXIS_WGS'] == 0 or row['DPR_Y_AXIS_WGS'] == 0 or row['ARV_X_AXIS_WGS'] == 0 or row['ARV_Y_AXIS_WGS'] == 0:
            return True

        # 좌표가 서울 범위를 벗어나는지 확인
        if not (SEOUl_MIN_LAT <= row['DPR_Y_AXIS_WGS'] <= SEOUl_MAX_LAT and SEOUl_MIN_LON <= row['DPR_X_AXIS_WGS'] <= SEOUl_MAX_LON):
            return False
        if not (SEOUl_MIN_LAT <= row['ARV_Y_AXIS_WGS'] <= SEOUl_MAX_LAT and SEOUl_MIN_LON <= row['ARV_X_AXIS_WGS'] <= SEOUl_MAX_LON):
            return False

        # 출발지와 도착지의 직선 거리 계산 (700미터 초과시 False)
        start_coords = (row['DPR_Y_AXIS_WGS'], row['DPR_X_AXIS_WGS'])
        end_coords = (row['ARV_Y_AXIS_WGS'], row['ARV_X_AXIS_WGS'])
        distance = geodesic(start_coords, end_coords).meters

        if distance > 700:
            return False

        return True

    # 유효한 데이터만 필터링
    df_cleaned = df[df.apply(valid_row, axis=1)]

    # 파일 저장
    df_cleaned.to_csv(output_file_path, index=False, encoding='utf-8-sig')
    print(f"파일 저장 완료: {output_file_path}")